<a href="https://colab.research.google.com/github/helen-mark/phonecall/blob/master/MCP_Call_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

!pip list | grep ollama

!ollama list

import subprocess, time
print("Запускаю Ollama сервер...")
subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(10)

import requests
try:
    response = requests.get("http://localhost:11434/api/tags", timeout=5)
    if response.status_code == 200:
        print("✅ Ollama сервер работает")
    else:
        print(f"⚠️  Ollama сервер отвечает с кодом {response.status_code}")
except:
    print("❌ Ollama сервер не отвечает")

print("Скачиваю модель mistral-nemo:12b...")
!ollama pull mistral-nemo:12b



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
ollama                                   0.6.1
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it
Запускаю Ollama сервер...
✅ Ollama сервер работает
Скачиваю модель mistral-nemo:12b...



In [8]:
CONFIG = {
    # GitHub
    "github_repo": "https://github.com/helen-mark/phonecall.git",
    "repo_name": 'phonecall',

    # Google Drive пути
    "drive_root": "/content/drive/MyDrive",
    "project_folder": "MCP_Call_Analytics",

    # Папки для данных
    "folders": {
        "audio_raw": "audio_raw",
        "audio_processed": "audio_processed",
        "csv_calls": "csv_calls",
        "saved_results": "saved_results",
        "logs": "logs"
    },

    # Модели
    "llm_model": "mistral-nemo:12b",

    # Настройки обработки
    "check_interval": 30,  # секунды между проверками новых файлов
    "auto_process": True   # Автоматически обрабатывать новые файлы
}

In [9]:
import os
# ====================
# 🚀 ЗАГРУЗКА КОДА ИЗ GITHUB РЕПОЗИТОРИЯ
# ====================

from google.colab import files
import sys
from google.colab import drive

print("📦 Загружаю проект из GitHub...")

repo_url = CONFIG["github_repo"]
repo_name = CONFIG["repo_name"]

!rm -rf {repo_name} 2>/dev/null || true

!git clone {repo_url}

%cd {repo_name+ '/colab'}

print("\n📁 Project structure:")
!find . -name "*.py" | head -20

project_path = os.getcwd()
sys.path.insert(0, project_path)
print(f"\n📍 Проект загружен в: {project_path}")
print(f"📦 Python path обновлен")

print("\n2️⃣ Монтирую Google Drive...")
drive.mount('/content/drive', force_remount=True)

base_path = os.path.join(CONFIG['drive_root'], CONFIG['project_folder'])

for folder_name, folder_path in CONFIG['folders'].items():
    full_path = os.path.join(base_path, folder_path)
    os.makedirs(full_path, exist_ok=True)
    print(f"   ✅ {folder_name}: {full_path}")

required_files = ["main.py", "interactive.py", "mcp_orchestrator.py"]
missing_files = []

for file in required_files:
    if os.path.exists(file):
        print(f"✅ Найден: {file}")
    else:
        missing_files.append(file)
        print(f"❌ Отсутствует: {file}")

if missing_files:
    print(f"\n⚠️  Отсутствуют файлы: {missing_files}")
    print("Проверьте структуру репозитория на GitHub")
else:
    print("\n🎉 Все файлы загружены успешно!")

# print("📤 ЗАГРУЗИТЕ CSV ФАЙЛЫ ЗВОНКОВ:")
# csv_files = files.upload()

# for filename in csv_files.keys():
#     if filename.endswith('.csv'):
#         os.rename(filename, f"/content/csv_calls/{filename}")
#         print(f"✅ CSV файл: {filename}")



📦 Загружаю проект из GitHub...
Cloning into 'phonecall'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 92 (delta 37), reused 70 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 293.12 KiB | 1.70 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/phonecall/colab/phonecall/colab

📁 Project structure:
./mcp_orchestrator.py
./colab_minimal.py
./colab_drive_setup.py
./interactive.py
./main.py

📍 Проект загружен в: /content/phonecall/colab/phonecall/colab
📦 Python path обновлен

2️⃣ Монтирую Google Drive...
Mounted at /content/drive
   ✅ audio_raw: /content/drive/MyDrive/MCP_Call_Analytics/audio_raw
   ✅ audio_processed: /content/drive/MyDrive/MCP_Call_Analytics/audio_processed
   ✅ csv_calls: /content/drive/MyDrive/MCP_Call_Analytics/csv_calls
   ✅ saved_results: /content/drive/MyDrive/MCP_Call_Analytics/saved_results
   ✅ logs: /content/drive/MyDrive/MCP_Call_

In [10]:
import json
import pandas as pd

class AudioFileMonitor:
    """Мониторит новые аудиофайлы в папке audio_raw"""

    def __init__(self, audio_raw_path, csv_path, log_path):
        self.audio_raw_path = audio_raw_path
        self.csv_path = csv_path
        self.log_path = log_path
        self.processed_files = self._load_processed_files()

        os.makedirs(log_path, exist_ok=True)
        self.log_file = os.path.join(log_path, "audio_processing.log")

    def _load_processed_files(self):
        """Загружает список уже обработанных файлов из CSV"""
        processed = set()

        # Проверяем CSV файлы
        if os.path.exists(self.csv_path):
            csv_files = [f for f in os.listdir(self.csv_path) if f.endswith('.csv')]
            for csv_file in csv_files:
                try:
                    df = pd.read_csv(os.path.join(self.csv_path, csv_file))
                    if 'source_file' in df.columns:
                        processed.update(df['source_file'].dropna().unique())
                except:
                    pass

        # # Также проверяем логи
        # log_file = os.path.join(self.log_path, "processed_audio.json")
        # if os.path.exists(log_file):
        #     with open(log_file, 'r') as f:
        #         processed.update(json.load(f))

        return processed

    def check_new_files(self):
        """Проверяет наличие новых аудиофайлов"""
        if not os.path.exists(self.audio_raw_path):
            return []

        # Получаем все аудиофайлы
        audio_extensions = ['.mp3', '.wav', '.m4a', '.flac', '.ogg']
        all_files = []

        for ext in audio_extensions:
            all_files.extend(Path(self.audio_raw_path).glob(f"*{ext}"))
            all_files.extend(Path(self.audio_raw_path).glob(f"*{ext.upper()}"))

        # Фильтруем новые файлы
        new_files = []
        for file_path in all_files:
            file_name = file_path.name
            if file_name not in self.processed_files:
                new_files.append(str(file_path))

        return new_files

    def log_processing(self, audio_file, status, details=""):
        """Логирует процесс обработки"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"{timestamp} | {audio_file} | {status} | {details}\n"

        # Записываем в лог-файл
        with open(self.log_file, 'a', encoding='utf-8') as f:
            f.write(log_entry)

        # Обновляем список обработанных файлов
        if status == "COMPLETED":
            self.processed_files.add(os.path.basename(audio_file))
            self._save_processed_files()

        print(f"📝 {log_entry.strip()}")

    def _save_processed_files(self):
        """Сохраняет список обработанных файлов"""
        log_file = os.path.join(self.log_path, "processed_audio.json")
        with open(log_file, 'w') as f:
            json.dump(list(self.processed_files), f)

# Инициализируем монитор
AUDIO_RAW_PATH = os.path.join(base_path, CONFIG['folders']['audio_raw'])
CSV_PATH = os.path.join(base_path, CONFIG['folders']['csv_calls'])
LOG_PATH = os.path.join(base_path, CONFIG['folders']['logs'])

monitor = AudioFileMonitor(AUDIO_RAW_PATH, CSV_PATH, LOG_PATH)

In [11]:
import sys
sys.path.append('.')

from interactive import enhanced_interactive_mode

enhanced_interactive_mode(
    _model="mistral-nemo:12b",
    csv_dir=CSV_PATH,
    results_dir="/content/saved_results",
    drive_path=None
)


╔══════════════════════════════════════════╗
║    🤖 АНАЛИТИК ЗВОНКОВ v3.0               ║
║    🚀 Google Drive Edition                ║
╚══════════════════════════════════════════╝
    
✅ Ollama подключен, модель: mistral-nemo:12b
📂 Загружаю данные из JSON файлов...
📂 Читаю данные из CSV файла: calls.csv
✅ Загружено 202 строк из CSV
✅ Преобразовано 202 записей звонков
📅 Диапазон дат: 01.10.2025 - 20.01.2026
🏷️  Уникальных тегов: 19
✅ Загружено 202 звонков

[Local] 📝 Вопрос (/? для помощи): на что жаловались клиенты в октябре 2025
🔍 Анализирую: 'на что жаловались клиенты в октябре 2025'

🔍 Анализирую запрос: 'на что жаловались клиенты в октябре 2025'
🤖 Создаю план анализа...
   📅 Период: октябрь 2025 года
   🏷️  Теги: низкое_качество_стирки_или_чистки, не_заменили_ковры_вовремя, клиент_хочет_добавить_ковры, клиент_хочет_меньше_ковров, погашение_долга, расторжение_договора, возобновление_услуг, долго_нет_ответа_на_заявку, лишняя_доставка, доставили_не_те_ковры, не_выставлен_вовремя_счет,